In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import os
# os.environ['MKL_NUM_THREADS'] = '1'
# os.environ['OMP_NUM_THREADS'] = '1'
# os.environ['MKL_DYNAMIC'] = 'FALSE'

In [2]:
import os
import sys
import time

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['GORDON_REPO_DIR'], 'utilities'))
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

from joblib import Parallel, delayed
from scipy.signal import fftconvolve

import bloscpack as bp
import blosc

dm = DataManager(stack='MD589', section=49)

print 'reading image ...',
t = time.time()
dm._load_image(versions=['gray'])
dm._generate_kernels()
print 'done in', time.time() - t, 'seconds'

reading image ... done in 0.000104904174805 seconds


In [70]:
xmin = dm.xmin
xmax = dm.xmin+8000
ymin = dm.ymin
ymax = dm.ymin+8000

In [68]:
xmin = dm.xmin
xmax = dm.xmax
ymin = dm.ymin
ymax = dm.ymax

In [76]:
mask = dm.mask[ymin:ymax+1, xmin:xmax+1]

perc = np.count_nonzero(mask) / float(mask.size)
sys.stderr.write('masked pixels = %.2f, ' % perc)


masked pixels = 0.92, 

In [71]:
def convolve_per_proc(i):

    pf = fftconvolve(dm.image[ymin-dm.max_kern_size : ymax+1+dm.max_kern_size, 
                              xmin-dm.max_kern_size : xmax+1+dm.max_kern_size], 
                     dm.kernels[i], 'same').astype(np.half)

    sys.stderr.write('filtered kernel %d\n'%i)

    return pf[dm.max_kern_size:-dm.max_kern_size, dm.max_kern_size:-dm.max_kern_size].flatten()


In [80]:
t = time.time()
sys.stderr.write('gabor filtering ...')

mask = dm.mask[ymin:ymax+1, xmin:xmax+1]

# f = convolve_per_proc(88)
# _ = Parallel(n_jobs=6)(delayed(convolve_per_proc)(i) for i in range(dm.n_kernel))
# features = Parallel(n_jobs=6)(delayed(convolve_per_proc)(i) for i in range(dm.n_kernel))
# features = Parallel(n_jobs=6, backend='threading')(delayed(convolve_per_proc)(i) for i in range(dm.n_kernel))
# features = Parallel(n_jobs=6, backend='threading')(delayed(convolve_per_proc)(i) for i in range(66, 99))
# features = Parallel(n_jobs=6)(delayed(convolve_per_proc)(i) for i in range(66, 99))
# features = Parallel(n_jobs=2, backend='threading')(delayed(convolve_per_proc)(i) for i in range(88, 90))
# features = Parallel(n_jobs=8)(delayed(convolve_per_proc)(i) for i in range(88, 99))

# sys.stderr.write('done in %f seconds\n' % (time.time() - t))

# t = time.time()

# sys.stderr.write('gabor filtering ...')

# features = Parallel(n_jobs=7, backend='threading')(delayed(convolve_per_proc)(i) for i in range(dm.n_kernel))
features = Parallel(n_jobs=7)(delayed(convolve_per_proc)(i) for i in range(dm.n_kernel))
# features = Parallel(n_jobs=7, backend='threading')(delayed(convolve_per_proc)(i) for i in range(88, 95))
# features = Parallel(n_jobs=8)(delayed(convolve_per_proc)(i) for i in range(88, 96))

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

gabor filtering ...filtered kernel 1
filtered kernel 6
filtered kernel 2
filtered kernel 5
filtered kernel 4
filtered kernel 0
filtered kernel 3
filtered kernel 7
filtered kernel 8
filtered kernel 10
filtered kernel 9
filtered kernel 11
filtered kernel 12
filtered kernel 13
filtered kernel 16
filtered kernel 15
filtered kernel 17
filtered kernel 14
filtered kernel 18
filtered kernel 19
filtered kernel 20
filtered kernel 23
filtered kernel 22
filtered kernel 21
filtered kernel 24
filtered kernel 25
filtered kernel 27
filtered kernel 26
filtered kernel 28
filtered kernel 29
filtered kernel 30
filtered kernel 31
filtered kernel 32
filtered kernel 33
filtered kernel 34
filtered kernel 36
filtered kernel 37
filtered kernel 38
filtered kernel 35
filtered kernel 39
filtered kernel 40
filtered kernel 41
filtered kernel 42
filtered kernel 43
filtered kernel 44
filtered kernel 45
filtered kernel 46
filtered kernel 47
filtered kernel 48
filtered kernel 49
filtered kernel 50
filtered kernel 51
fil

In [78]:
t = time.time()
sys.stderr.write('transpose ...')

features = np.asarray(features).T  # n x 99

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

t = time.time()
sys.stderr.write('mask ...')

features_masked = features[mask.flat]

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

transpose ...done in 0.505933 seconds
mask .../oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:11: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
done in 2.387227 seconds


In [5]:
# very slow
# https://pythonhosted.org/joblib/parallel.html#writing-parallel-computation-results-in-shared-memory

def convolve_per_proc(output, i):

    pf = fftconvolve(dm.image[ymin-dm.max_kern_size : ymax+1+dm.max_kern_size, 
                              xmin-dm.max_kern_size : xmax+1+dm.max_kern_size], 
                     dm.kernels[i], 'same').astype(np.half)

    sys.stderr.write('filtered kernel %d\n'%i)

    output[i] = pf[dm.max_kern_size:-dm.max_kern_size, dm.max_kern_size:-dm.max_kern_size]
    
import tempfile
import shutil
import os

from joblib import Parallel, delayed
from joblib import load, dump

folder = tempfile.mkdtemp()
features_name = os.path.join(folder, 'features')
features = np.memmap(features_name, dtype=np.half, shape=(dm.n_kernel, ymax-ymin+1, xmax-xmin+1), mode='w+')

t = time.time()
sys.stderr.write('gabor filtering ...')

# Parallel(n_jobs=6, backend='threading')(delayed(convolve_per_proc)(features, i) for i in range(dm.n_kernel))
Parallel(n_jobs=11, backend='threading')(delayed(convolve_per_proc)(features, i) for i in range(88,99))

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

try:
    shutil.rmtree(folder)
except:
    print("Failed to delete: " + folder)

In [ ]:
t = time.time()
sys.stderr.write('gabor filtering ...')

mask = dm.mask[ymin:ymax+1, xmin:xmax+1]

features = Parallel(n_jobs=8)(delayed(convolve_per_proc)(i) for i in range(dm.n_kernel))

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

t = time.time()
sys.stderr.write('transpose ...')

features = np.asarray(features).T  # n x 99

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

t = time.time()
sys.stderr.write('mask ...')

features_masked = features[mask.flat]

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

In [5]:
# assert dm.ymin-dm.max_kern_size >= 0 and dm.xmin-dm.max_kern_size >= 0 and \
#         dm.ymax+1+dm.max_kern_size <= dm.image_height and dm.xmax+1+dm.max_kern_size <= dm.image_width, \
#         'Not all pixels within the mask have value from the largest kernel'

def convolve_per_proc(i):
    pf = fftconvolve(dm.image[dm.ymin-dm.max_kern_size : dm.ymax+1+dm.max_kern_size, 
                              dm.xmin-dm.max_kern_size : dm.xmax+1+dm.max_kern_size], 
                     dm.kernels[i], 'same').astype(np.half)
    sys.stderr.write('filtered kernel %d\n'%i)

#     bp.pack_ndarray_file(pf[dm.max_kern_size:-dm.max_kern_size, dm.max_kern_size:-dm.max_kern_size].copy(), 
#                          os.environ['GORDON_RESULT_DIR']+'/feature_%03d.bp'%i)
    
#     return pf[dm.max_kern_size:-dm.max_kern_size, dm.max_kern_size:-dm.max_kern_size]


In [ ]:
feature_0 = pf[dm.max_kern_size:-dm.max_kern_size, dm.max_kern_size:-dm.max_kern_size]
plt.matshow(feature_0);

In [ ]:
# with open('/home/yuncong/CSHL_data_results/feature_000.bp', 'wb') as f:
#     f.write(blosc.pack_array(feature_0))

t = time.time()
# features_0_load = bp.unpack_ndarray_str(bp.pack_ndarray_str(feature_0.copy()))
features_0_load = blosc.unpack_array(blosc.pack_array(feature_0))
print time.time() - t

In [ ]:
# with open('/home/yuncong/CSHL_data_results/feature_000.bp', 'rb') as f:
#     s = f.read()

In [ ]:
t = time.time()
print 'gabor filtering...',

for i in range(dm.n_kernel):
    convolve_per_proc(i)

print 'done in', time.time() - t, 'seconds'

filtered kernel 0
filtered kernel 1
filtered kernel 2
filtered kernel 3
filtered kernel 4
filtered kernel 5


In [ ]:
t = time.time()
print 'gabor filtering...',

# filtered = Parallel(n_jobs=4)(delayed(convolve_per_proc)(i) for i in range(dm.n_kernel))
# features = np.asarray(filtered)
# del filtered

Parallel(n_jobs=4)(delayed(convolve_per_proc)(i) for i in range(dm.n_kernel))

print 'done in', time.time() - t, 'seconds'

In [ ]:
dm.save_pipeline_result(features, 'features')

In [ ]:
t = time.time()
print 'load filtered values ...',

features = []
for i in range(dm.n_kernel):
    a = bp.unpack_ndarray_file(os.environ['GORDON_RESULT_DIR']+'/feature_%03d.bp'%i).reshape((-1,)) 
    features.append(a)
    
print 'done in', time.time() - t, 'seconds'

In [ ]:
del features

In [ ]:
t = time.time()
print 'load filtered values ...',

features = [bp.unpack_ndarray_file(os.environ['GORDON_RESULT_DIR']+'/feature_%03d.bp'%i).reshape((-1,)) 
                     for i in range(dm.n_kernel)]

print 'done in', time.time() - t, 'seconds'

In [ ]:
del features

In [ ]:
features = features.T

In [ ]:
def rotate_features(fs, j):
    features_tabular = fs.reshape((fs.shape[0], dm.n_freq, dm.n_angle))
    max_angle_indices = features_tabular.max(axis=1).argmax(axis=-1)
    features_rotated = np.reshape([np.roll(features_tabular[i], -ai, axis=-1) 
                               for i, ai in enumerate(max_angle_indices)], (fs.shape[0], dm.n_freq * dm.n_angle))

    del features_tabular, max_angle_indices
    
#     print features_rotated.shape
#     bp.pack_ndarray_file(features_rotated, os.environ['GORDON_RESULT_DIR']+'/featureRotated_batch_%03d.bp'%j)

    return features_rotated

In [ ]:
features.shape[0]

In [ ]:
t = time.time()
print 'rotating features ...',

items_per_job = 100
features_rotated = Parallel(n_jobs=8)(delayed(rotate_features)(fs, i) 
                   for i, fs in enumerate(np.array_split(features, features.shape[0]/items_per_job)))

print 'done in', time.time() - t, 'seconds'

In [ ]:
del features_rotated

In [ ]:
features_rotated = np.vstack(features_rotated)

In [ ]:
features_rotated.shape

In [ ]:
t = time.time()
print 'load filtered values ...',

items_per_job = 100
features_rotated = Parallel(n_jobs=8)(delayed(rotate_features)(fs, i) 
                   for i, fs in enumerate(np.array_split(features[:10000000], 10000000/items_per_job)))
# features_rotated = np.vstack(features_rotated)

print 'done in', time.time() - t, 'seconds'

In [ ]:
features_rotated = np.vstack(features_rotated)

In [ ]:
features_rotated.shape

In [ ]:
from skimage.util import pad

approx_bg_intensity = dm.image[10:20, 10:20].mean()
# approx_bg_intensity = 0

masked_image = dm.image.copy()
masked_image[~dm.mask] = approx_bg_intensity

# padded_image = pad(masked_image, max_kern_size, 'constant', constant_values=approx_bg_intensity)
padded_image = pad(masked_image, dm.max_kern_size, 'linear_ramp', end_values=approx_bg_intensity)

# plt.imshow(padded_image, cm.Greys_r)
# plt.show()

# display(padded_image)

In [ ]:
import time
b = time.time()

# try:
#     features = dm.load_pipeline_result('features', 'npy')
    
# except Exception as e:

def convolve_per_proc(i):
    return fftconvolve(padded_image, dm.kernels[i], 'same').astype(np.half)

padded_filtered = Parallel(n_jobs=16)(delayed(convolve_per_proc)(i) 
                        for i in range(dm.n_kernel))

filtered = [f[dm.max_kern_size:-dm.max_kern_size, dm.max_kern_size:-dm.max_kern_size] for f in padded_filtered]

#     features = np.empty((dm.image_height, dm.image_width, n_kernel), dtype=np.half)
#     for i in range(n_kernel):
#         features[...,i] = filtered[i]

features = np.empty((dm.n_kernel, dm.image_height, dm.image_width), dtype=np.half)
for i in range(dm.n_kernel):
    features[i, ...] = filtered[i]

del filtered

dm.save_pipeline_result(features, 'features', 'npy')

print 'gabor filtering', time.time() - b

In [ ]:
# # visualize a slice of feature responses

# cropped_response = features[-1]
# plt.matshow(cropped_response, cmap=cm.coolwarm)
# plt.colorbar()
# plt.show()

In [ ]:
# from skimage.exposure import rescale_intensity
# cropped_response_vis = rescale_intensity(cropped_response, out_range=(0, 255))
# cropped_response_vis[~dm.mask] = 127
# display(cropped_response_vis.astype(np.uint8))